# Premiers flocons

Ce petit script parcourt un fichier de MétéoSuisse constitué d'une ligne par jour depuis 1980, avec un 1 pour "flocons observés" et un 0 pour "pas de flocons observés". La date du 11 août 2008 est une erreur (observation de grêle notée comme observation de neige).

In [1]:
import pandas as pd
df = pd.read_csv('gve_neige.txt', sep=' ')

In [2]:
del df['Unnamed: 7']
df.columns = ['STA','JAHR','MO','TG','HH','MM','908']

In [4]:
df_neige = df[df['908'] > 0]

In [5]:
last_year = 1979
ignore_year = 0
date_collection = []

for index, row in df_neige.iterrows():
    if row['JAHR'] > last_year:
        
        if row['JAHR'] > last_year + 1: # on a sauté une année
            if row['JAHR'] != ignore_year:
                print('Premiers flocons tardifs pour l’année', row['JAHR']-1)
                date_collection.append((row['JAHR']-1, row['MO'], row['TG']))
                ignore_year = row['JAHR']
                
        if row['MO'] > 6: # cas standard
            last_year = row['JAHR']
            date_collection.append((row['JAHR'], row['MO'], row['TG']))
        else:
            pass
        
date_collection

Premiers flocons tardifs pour l’année 1989
Premiers flocons tardifs pour l’année 2006


[(1980, 11, 4),
 (1981, 11, 29),
 (1982, 11, 16),
 (1983, 12, 3),
 (1984, 12, 26),
 (1985, 11, 12),
 (1986, 12, 19),
 (1987, 12, 8),
 (1988, 11, 20),
 (1989, 1, 28),
 (1990, 11, 22),
 (1991, 12, 20),
 (1992, 12, 12),
 (1993, 11, 20),
 (1994, 12, 26),
 (1995, 11, 18),
 (1996, 11, 18),
 (1997, 12, 2),
 (1998, 11, 23),
 (1999, 11, 16),
 (2000, 12, 29),
 (2001, 11, 9),
 (2002, 12, 10),
 (2003, 11, 10),
 (2004, 11, 11),
 (2005, 11, 24),
 (2006, 1, 23),
 (2007, 11, 13),
 (2008, 8, 11),
 (2009, 12, 13),
 (2010, 11, 26),
 (2011, 12, 17),
 (2012, 10, 27),
 (2013, 11, 20),
 (2014, 12, 26),
 (2015, 11, 21),
 (2016, 11, 7)]

In [6]:
df_export = pd.DataFrame(date_collection)

In [7]:
df_export.columns = ('Année', 'Mois', 'Jour')
df_export.head()

,Année,Mois,Jour
0,1980,11,4
1,1981,11,29
2,1982,11,16
3,1983,12,3
4,1984,12,26


### On ajoute les mois en toutes lettres et un timestamp (pour les graphiques)

In [8]:
for i, row in df_export.iterrows():
    # zfill + format, c'est pas beau, j'avoue
    timestamp = "{}-{}-{}".format(row['Année'], str(row['Mois']).zfill(2), str(row['Jour']).zfill(2))
    df_export.set_value(i, 'timestamp', timestamp)
    
    number = row['Mois']*30 + row['Jour']
    if row['Mois'] < 6:
        number += 365 # annee d'après
    df_export.set_value(i, 'number', number)
    

In [12]:
def get_month(value):
    months = ['janvier', 'février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'septembre', 'octobre', 'novembre', 'décembre']
    return months[int(value)-1]
df_export['Mois_str'] = df_export['Mois'].apply(get_month)

In [11]:
df_export.to_excel('premiers_flocons.xls', index=False)